In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(style="darkgrid")


In [2]:
from tensorflow.keras.models import load_model

import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
# tf.test.is_gpu_available()
from tensorflow.keras import Input,Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Concatenate, Conv2D,Conv1D,Convolution1D,\
Flatten, BatchNormalization,Input,Dropout
from tensorflow.keras.layers import LeakyReLU, ReLU, LayerNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adadelta, Adam, SGD
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, MAE, KLD, MSLE, MSE

In [3]:
# load model
model = load_model('my_model_2500.h5') 

In [10]:
theta1, theta2 = 15, 36
sr = 2048
std_rm, loss_rm = test_batch()
print(std_rm,'\n', loss_rm)

[0.04153961 0.0365617  0.02980198 0.02637253 0.02835643 0.02174973
 0.02544906] 
 [0.68060652 0.6754395  0.70497565 0.72493936 0.73789278 0.7594866
 0.8044965 ]


In [9]:
def test_batch():
#     RM = [0.32, 0.42, 0.52, 0.62, 0.72, 0.82, 0.92]
    RM = [0.66, 0.67, 0.69, 0.72, 0.75, 0.78, 0.82]
    ALPHA=[1.2, 10.2, 20.2, 50.2, 100.2, 150.2, 200.2, 250.2]
    RES = np.zeros((len(RM),len(ALPHA)))
    for ir, rm in enumerate(RM):
        for ia, alpha in enumerate(ALPHA):
            test_dop = test_simple(theta1, theta2,alpha,rm, noise=0)[2]
            pred_rm = model.predict(test_dop[None,])
            
            RES[ir][ia] = pred_rm
    
    loss_rm = np.mean(RES, axis=1)
    std_rm = np.std(RES, axis=1)
    return std_rm, loss_rm

def test_generateData(alpha,rm,im =0.0,N_ta = 2**13-1):
    origin_path = os.getcwd()
#     os.chdir(r"C:\Users\beidongjiedeguang\Desktop\测试文件夹")
    os.chdir(r"C:\Users\Administrator\OneDrive\Mie测试文件夹")
    exeName = "Mie_Calculation.exe"
    txtName = '输入参数文本.txt'

    Ta_min, Ta_max = 0,180
    with open(txtName,'w') as f1:
        f1.write('{}\n{:.3f}\t{:.3f}\n{}\t{:.2f}\t{:.2f}'
                .format(alpha,rm,im,N_ta,Ta_min,Ta_max))

    os.system(exeName)
    dataName = "散射光强角分布.txt"
    # 没想到c写出来的不是utf-8编码的，所以这里encoding='gbk'，后面这个delimiter随便要不要
    data = pd.read_csv(dataName,sep ='\t',engine='python',encoding='gbk',delimiter='\s+')
    os.remove(dataName)
    os.chdir(origin_path)
    return data

def gen(alpha = 900, rm = 1.7, im= 0.0, N_ta = 2**13-1):
    data = test_generateData(alpha,rm,im, N_ta = N_ta)
    theta = data['散射角'].to_numpy()
    i1, i2, DOP = data['I1'].to_numpy(), data['I2'].to_numpy(), data['可视度'].to_numpy()
    return i1[...,None], i2[...,None], DOP[...,None]

def test_simple(theta1, theta2, alpha, rm, noise=False):
    test_i1, test_i2, test_dop = gen(alpha = alpha, rm = rm, N_ta = sr-1) # 
    
    d_the = (sr-1)/180
    th1, th2 = int(theta1*d_the), int(theta2*d_the)
    SR = th2-th1
    test_i1, test_i2, test_dop = test_i1[th1:th2], test_i2[th1:th2], test_dop[th1:th2]
    
    if noise:
        noise = np.random.rand(SR, 1) * 0.05
        noise_dop = (np.random.rand(SR, 1)-0.5) * 0.05
        test_i1, test_i2, = test_i1+noise, test_i2+noise
        test_dop = test_dop + noise_dop
    return test_i1, test_i2, test_dop 

